# Machine Learning Engineer Nanodegree
## Unsupervised Learning
## Project 3: Creating Customer Segments

Welcome to the third project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

## Getting Started

In this project, you will analyze a dataset containing data on various customers' annual spending amounts (reported in *monetary units*) of diverse product categories for internal structure. One goal of this project is to best describe the variation in the different types of customers that a wholesale distributor interacts with. Doing so would equip the distributor with insight into how to best structure their delivery service to meet the needs of each customer.

The dataset for this project can be found on the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Wholesale+customers). For the purposes of this project, the features `'Channel'` and `'Region'` will be excluded in the analysis — with focus instead on the six product categories recorded for customers.

Run the code block below to load the wholesale customers dataset, along with a few of the necessary Python libraries required for this project. You will know the dataset loaded successfully if the size of the dataset is reported.

In [39]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import renders as rs
from IPython.display import display # Allows the use of display() for DataFrames

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Load the wholesale customers dataset
# try:
#     data = pd.read_csv("customers.csv")
#     data.drop(['Region', 'Channel'], axis = 1, inplace = True)
#     print("Wholesale customers dataset has {} samples with {} features each.".format(*data.shape))
# except ValueError:
#     print("Dataset could not be loaded. Is the dataset missing?")

SyntaxError: Missing parentheses in call to 'print' (renders.py, line 86)

In [40]:
df = pd.read_csv("customers.csv")
df.drop(['Region', 'Channel'], axis = 1, inplace = True)
df.head(1)

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
0,12669,9656,7561,214,2674,1338


## Data Exploration
In this section, you will begin exploring the data through visualizations and code to understand how each feature is related to the others. You will observe a statistical description of the dataset, consider the relevance of each feature, and select a few sample data points from the dataset which you will track through the course of this project.

Run the code block below to observe a statistical description of the dataset. Note that the dataset is composed of six important product categories: **'Fresh'**, **'Milk'**, **'Grocery'**, **'Frozen'**, **'Detergents_Paper'**, and **'Delicatessen'**. Consider what each category represents in terms of products you could purchase.

In [41]:
# Display a description of the dataset
df.describe()

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000
mean,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455
std,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937
min,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000
25%,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000
50%,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000
75%,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000
max,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000


### Implementation: Selecting Samples
To get a better understanding of the customers and how their data will transform through the analysis, it would be best to select a few sample data points and explore them in more detail. In the code block below, add **three** indices of your choice to the `indices` list which will represent the customers to track. It is suggested to try different sets of samples until you obtain customers that vary significantly from one another.

In [42]:
# TODO: Select three indices of your choice you wish to sample from the dataset
indices = [0,77,99]

# Create a DataFrame of the chosen samples
df_samples = pd.DataFrame(df.loc[indices], columns = df.keys()).reset_index(drop = True)
print("Chosen samples of wholesale customers dataset:")
df_samples

Chosen samples of wholesale customers dataset:


,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
0,12669,9656,7561,214,2674,1338
1,12205,12697,28540,869,12034,1009
2,9658,2182,1909,5639,215,323


### Question 1
Consider the total purchase cost of each product category and the statistical description of the dataset above for your sample customers.  
*What kind of establishment (customer) could each of the three samples you've chosen represent?*  
**Hint:** Examples of establishments include places like markets, cafes, and retailers, among many others. Avoid using names for establishments, such as saying *"McDonalds"* when describing a sample customer as a restaurant.

In [43]:
for index, row in df_samples.iterrows():
    print(df_samples.iloc[index] / df_samples.sum(axis=1)[index] *100)

Fresh               37.139423
Milk                28.306754
Grocery             22.165220
Frozen               0.627345
Detergents_Paper     7.838884
Delicatessen         3.922373
Name: 0, dtype: float64
Fresh               18.120676
Milk                18.851145
Grocery             42.373133
Frozen               1.290198
Detergents_Paper    17.866793
Delicatessen         1.498055
Name: 1, dtype: float64
Fresh               48.469337
Milk                10.950517
Grocery              9.580448
Frozen              28.299709
Detergents_Paper     1.078992
Delicatessen         1.620998
Name: 2, dtype: float64


**Answer:**
Based on the above results, which show what percent of the spending for the customer was spent on which product cateogry, it gives use some intuition about what type of store the customer is. 

For the first sample, it is clear that this customers spends the majority on  Fresh, Milk, Grocery, but also spend a small amount on Frozen and Deteregents_Papper, which indicated that this customer is likely a coffee shop or cafe.

For the second sample, the majority is spent on Grocery(42% of total), but also alot spent on Fresh, Milk, Detergent (all around 17-18% of total). This is likely that this customer is a grocery store. 

For the third sample the majority is spent on fresh(48%), with alot spent on Frozen(28%) and Milk (10%). This place would likely be a grocery store or market. 

### Implementation: Feature Relevance
One interesting thought to consider is if one (or more) of the six product categories is actually relevant for understanding customer purchasing. That is to say, is it possible to determine whether customers purchasing some amount of one category of products will necessarily purchase some proportional amount of another category of products? We can make this determination quite easily by training a supervised regression learner on a subset of the data with one feature removed, and then score how well that model can predict the removed feature.

In the code block below, you will need to implement the following:
 - Assign `new_data` a copy of the data by removing a feature of your choice using the `DataFrame.drop` function.
 - Use `sklearn.cross_validation.train_test_split` to split the dataset into training and testing sets.
   - Use the removed feature as your target label. Set a `test_size` of `0.25` and set a `random_state`.
 - Import a decision tree regressor, set a `random_state`, and fit the learner to the training data.
 - Report the prediction score of the testing set using the regressor's `score` function.

In [44]:
# TODO: Make a copy of the DataFrame, using the 'drop' function to drop the given feature
X = new_data = df.drop(['Frozen'], axis = 1)
y = df['Frozen']

# TODO: Split the data into training and testing sets using the given feature as the target
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)

# TODO: Create a decision tree regressor and fit it to the training set
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X_train,y_train)

# TODO: Report the score of the prediction using the testing set
score = regressor.score(X_test,y_test)
print(score)

-0.278249148824


In [45]:
for columnName in df.columns.values:
    print(columnName)

Fresh
Milk
Grocery
Frozen
Detergents_Paper
Delicatessen


### Question 2
*Which feature did you attempt to predict? What was the reported prediction score? Is this feature is necessary for identifying customers' spending habits?*  
**Hint:** The coefficient of determination, `R^2`, is scored between 0 and 1, with 1 being a perfect fit. A negative `R^2` implies the model fails to fit the data.

**Answer:**
I chose the feature of frozen because intuitivly I thought that if a customer purchased fresh or groceries, thier buying of frozen would be low. But the score is approximately -0.27, which indicates that this does not fit the data well. Other models should be looked, as this one does not fit the model well. 

The F1 Score of Frozen, indicated that this is NOT a relevant feature for out examination. Below I quickly got F1 Scores for other features to see which feature I should have chossen.  

In [46]:
def checkF1():
    for feature in df.columns.values:
        X = new_data = df.drop([feature], axis = 1)
        y = df[feature]

        # TODO: Split the data into training and testing sets using the given feature as the target
        from sklearn.cross_validation import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)

        # TODO: Create a decision tree regressor and fit it to the training set
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state=0)
        regressor.fit(X_train,y_train)

        # TODO: Report the score of the prediction using the testing set
        score = regressor.score(X_test,y_test)
        print("Feature: ", feature, " F1 Score: ", score)

In [47]:
checkF1()

Feature:  Fresh  F1 Score:  -0.333070533605
Feature:  Milk  F1 Score:  0.173438009379
Feature:  Grocery  F1 Score:  0.699248196675
Feature:  Frozen  F1 Score:  -0.278249148824
Feature:  Detergents_Paper  F1 Score:  0.348777454691
Feature:  Delicatessen  F1 Score:  -11.0236279005


By looking at the above results, we can see that Grocery, Detergents, Milk are all good features too look at, while Fresh, Frozen and Delicateseen are not so great. 

### Visualize Feature Distributions
To get a better understanding of the dataset, we can construct a scatter matrix of each of the six product features present in the data. If you found that the feature you attempted to predict above is relevant for identifying a specific customer, then the scatter matrix below may not show any correlation between that feature and the others. Conversely, if you believe that feature is not relevant for identifying a specific customer, the scatter matrix might show a correlation between that feature and another feature in the data. Run the code block below to produce a scatter matrix.

In [48]:
# Produce a scatter matrix for each pair of features in the data
pd.scatter_matrix(df, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

### Question 3
*Are there any pairs of features which exhibit some degree of correlation? Does this confirm or deny your suspicions about the relevance of the feature you attempted to predict? How is the data for those features distributed?*  
**Hint:** Is the data normally distributed? Where do most of the data points lie? 

**Answer:**
My initial prediction was off, and this seems like Grocery and detergence_papper are the lesat correlated with one another.  The dist for all 6 features seems skewed to the right. Alot of the data is close to 0 if you ignore the outliers, if we remove outliers, we could consider it normal.

## Data Preprocessing
In this section, you will preprocess the data to create a better representation of customers by performing a scaling on the data and detecting (and optionally removing) outliers. Preprocessing data is often times a critical step in assuring that results you obtain from your analysis are significant and meaningful.

### Implementation: Feature Scaling
If data is not normally distributed, especially if the mean and median vary significantly (indicating a large skew), it is most [often appropriate](http://econbrowser.com/archives/2014/02/use-of-logarithms-in-economics) to apply a non-linear scaling — particularly for financial data. One way to achieve this scaling is by using a [Box-Cox test](http://scipy.github.io/devdocs/generated/scipy.stats.boxcox.html), which calculates the best power transformation of the data that reduces skewness. A simpler approach which can work in most cases would be applying the natural logarithm.

In the code block below, you will need to implement the following:
 - Assign a copy of the data to `log_data` after applying a logarithm scaling. Use the `np.log` function for this.
 - Assign a copy of the sample data to `log_samples` after applying a logrithm scaling. Again, use `np.log`.

In [49]:
# TODO: Scale the data using the natural logarithm
log_data = np.log(df)

# TODO: Scale the sample data using the natural logarithm
log_samples = np.log(df_samples)

# Produce a scatter matrix for each pair of newly-transformed features
pd.scatter_matrix(log_data, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

### Observation
After applying a natural logarithm scaling to the data, the distribution of each feature should appear much more normal. For any pairs of features you may have identified earlier as being correlated, observe here whether that correlation is still present (and whether it is now stronger or weaker than before).

Run the code below to see how the sample data has changed after having the natural logarithm applied to it.

In [50]:
# Display the log-transformed sample data
print(log_samples)

      Fresh      Milk    Grocery    Frozen  Detergents_Paper  Delicatessen
0  9.446913  9.175335   8.930759  5.365976          7.891331      7.198931
1  9.409601  9.449121  10.259062  6.767343          9.395491      6.916715
2  9.175542  7.687997   7.554335  8.637462          5.370638      5.777652


### Implementation: Outlier Detection
Detecting outliers in the data is extremely important in the data preprocessing step of any analysis. The presence of outliers can often skew results which take into consideration these data points. There are many "rules of thumb" for what constitutes an outlier in a dataset. Here, we will use [Tukey's Method for identfying outliers](http://datapigtechnologies.com/blog/index.php/highlighting-outliers-in-your-data-with-the-tukey-method/): An *outlier step* is calculated as 1.5 times the interquartile range (IQR). A data point with a feature that is beyond an outlier step outside of the IQR for that feature is considered abnormal.

In the code block below, you will need to implement the following:
 - Assign the value of the 25th percentile for the given feature to `Q1`. Use `np.percentile` for this.
 - Assign the value of the 75th percentile for the given feature to `Q3`. Again, use `np.percentile`.
 - Assign the calculation of an outlier step for the given feature to `step`.
 - Optionally remove data points from the dataset by adding indices to the `outliers` list.

**NOTE:** If you choose to remove any outliers, ensure that the sample data does not contain any of these points!  
Once you have performed this implementation, the dataset will be stored in the variable `good_data`.

In [51]:
# For each feature find the data points with extreme high or low values

outliers = []

for feature in log_data.keys():
    print(feature)
    # TODO: Calculate Q1 (25th percentile of the data) for the given feature
    Q1 = np.percentile(log_data[feature], 25)
    print('Q1') 
    print(Q1)
    
    # TODO: Calculate Q3 (75th percentile of the data) for the given feature
    Q3 = np.percentile(log_data[feature], 75)
    print('Q3') 
    print(Q3)
    
    # TODO: Use the interquartile range to calculate an outlier step (1.5 times the interquartile range)
    step = 1.5*(Q3 - Q1)
    print('Step')
    print(step)
    
    overRange = log_data[feature] > (Q3+step)
    underRange = log_data[feature] < (Q1-step)
    outlier_in_feature = log_data[underRange | overRange]
    
    # Display the outliers
    print("Data points considered outliers for the feature '{}':".format(feature))
    print(outlier_in_feature)
    
    for index, row in outlier_in_feature.iterrows():
        outliers.append(index)
    
# OPTIONAL: Select the indices for data points you wish to remove
from itertools import groupby

#### Test to check for Dupliacate Outliers

# print("Duplicate Check")
# x = [1,2,2,2,3,4,5,6,6,7]
# print([key for key,group in groupby(x) if len(list(group)) > 1])

#### Done Test

print([key for key,group in groupby(outliers) if len(list(group)) > 1])

#outliers = np.unique(outliers)


# Remove the outliers, if any were specified
good_data = log_data.drop(log_data.index[outliers]).reset_index(drop = True)

Fresh
Q1
8.04805870221
Q3
9.73706394795
Step
2.53350786861
Data points considered outliers for the feature 'Fresh':
        Fresh       Milk    Grocery    Frozen  Detergents_Paper  Delicatessen
65   4.442651   9.950323  10.732651  3.583519         10.095388      7.260523
66   2.197225   7.335634   8.911530  5.164786          8.151333      3.295837
81   5.389072   9.163249   9.575192  5.645447          8.964184      5.049856
95   1.098612   7.979339   8.740657  6.086775          5.407172      6.563856
96   3.135494   7.869402   9.001839  4.976734          8.262043      5.379897
128  4.941642   9.087834   8.248791  4.955827          6.967909      1.098612
171  5.298317  10.160530   9.894245  6.478510          9.079434      8.740337
193  5.192957   8.156223   9.917982  6.865891          8.633731      6.501290
218  2.890372   8.923191   9.629380  7.158514          8.475746      8.759669
304  5.081404   8.917311  10.117510  6.424869          9.374413      7.787382
305  5.493061   9.468001  

### Question 4
*Are there any data points considered outliers for more than one feature based on the definition above? Should these data points be removed from the dataset? If any data points were added to the `outliers` list to be removed, explain why.* 

**Answer:**
Based on the information in the question given, outliers were removed based on the the step. Took the 75th interquartile range and the 25th interquartile range diffence, and times it by 1.5 , if any data points per feature were outside of this range, they were removed from the data set as they are considerd outliers, and might affect this excercise negatievly.

I have chosen to remove outliers, that are not duplicates. Since I found none in the data set, I just removed ALL the outliers. My intuiition in NOT to remove the duplicate outliers, is that if a outlier was seen twice or more, then it is not an outlier, but is really a true point that would be seen by data set. 

## Feature Transformation
In this section you will use principal component analysis (PCA) to draw conclusions about the underlying structure of the wholesale customer data. Since using PCA on a dataset calculates the dimensions which best maximize variance, we will find which compound combinations of features best describe customers.

### Implementation: PCA

Now that the data has been scaled to a more normal distribution and has had any necessary outliers removed, we can now apply PCA to the `good_data` to discover which dimensions about the data best maximize the variance of features involved. In addition to finding these dimensions, PCA will also report the *explained variance ratio* of each dimension — how much variance within the data is explained by that dimension alone. Note that a component (dimension) from PCA can be considered a new "feature" of the space, however it is a composition of the original features present in the data.

In the code block below, you will need to implement the following:
 - Import `sklearn.decomposition.PCA` and assign the results of fitting PCA in six dimensions with `good_data` to `pca`.
 - Apply a PCA transformation of the sample log-data `log_samples` using `pca.transform`, and assign the results to `pca_samples`.

In [52]:
from sklearn.decomposition import PCA
# TODO: Apply PCA by fitting the good data with the same number of dimensions as features
pca = PCA(n_components = 6)
pca.fit(good_data)

# TODO: Transform the sample log-data using the PCA fit above
pca_samples = pca.transform(log_samples)
print(pca.explained_variance_ratio_)

# Generate PCA results plot
pca_results = rs.pca_results(good_data, pca)

[ 0.49930484  0.22594807  0.10488309  0.09781761  0.04879109  0.0232553 ]


NameError: name 'rs' is not defined

### Question 5
*How much variance in the data is explained* ***in total*** *by the first and second principal component? What about the first four principal components? Using the visualization provided above, discuss what the first four dimensions best represent in terms of customer spending.*  
**Hint:** A positive increase in a specific dimension corresponds with an *increase* of the *positive-weighted* features and a *decrease* of the *negative-weighted* features. The rate of increase or decrease is based on the indivdual feature weights.

In [53]:
print(pca.explained_variance_ratio_[0]*100)
print(pca.explained_variance_ratio_[1]*100)
print(pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1])

49.9304838075
22.5948066791
0.725252904866


In [54]:
print(pca.explained_variance_ratio_[3]*100)
print(pca.explained_variance_ratio_[4]*100)
print(pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1]+pca.explained_variance_ratio_[3]+pca.explained_variance_ratio_[4])

9.78176110491
4.8791091227
0.871861607142


**Answer:**
By looking at the "explained_variance_ratio" attribute we can see which contributes the most. The first attribute is contributing to almost half the variance, approximatley  50% of it, and the second ctrontibure 22%. Combined the first and second contribute to most of the varience being .72.

The next components (3,4) contribute significantly less at 9.78 and 4.89 repectively. They still play a part, but are not nearly as large of a factar as 1 and 2. The total for the first four components is 0.87.

Given the figure above, the first dimension has greater importance on Detergencts, Grocery and Milk, which seem to be most correlated. The second dimesnion lessens the impact of these, and puts more importance of Fresh, Frozen , and Deli. The third dimension looks like low fresh and high deli are going together. The 4th is similar to the thiird, in high deli and low fresh are seen together. 

### Observation
Run the code below to see how the log-transformed sample data has changed after having a PCA transformation applied to it in six dimensions. Observe the numerical value for the first four dimensions of the sample points. Consider if this is consistent with your initial interpretation of the sample points.

In [55]:
# Display sample log-data after having a PCA transformation applied
print(pd.DataFrame(np.round(pca_samples, 4), columns = pca_results.index.values))

NameError: name 'pca_results' is not defined

### Implementation: Dimensionality Reduction
When using principal component analysis, one of the main goals is to reduce the dimensionality of the data — in effect, reducing the complexity of the problem. Dimensionality reduction comes at a cost: Fewer dimensions used implies less of the total variance in the data is being explained. Because of this, the *cumulative explained variance ratio* is extremely important for knowing how many dimensions are necessary for the problem. Additionally, if a signifiant amount of variance is explained by only two or three dimensions, the reduced data can be visualized afterwards.

In the code block below, you will need to implement the following:
 - Assign the results of fitting PCA in two dimensions with `good_data` to `pca`.
 - Apply a PCA transformation of `good_data` using `pca.transform`, and assign the reuslts to `reduced_data`.
 - Apply a PCA transformation of the sample log-data `log_samples` using `pca.transform`, and assign the results to `pca_samples`.

In [56]:
# TODO: Apply PCA by fitting the good data with only two dimensions
pca = PCA(n_components=2)
pca.fit(good_data)

# TODO: Transform the good data using the PCA fit above
reduced_data = pca.transform(good_data)

# TODO: Transform the sample log-data using the PCA fit above
pca_samples = pca.transform(log_samples)

# Create a DataFrame for the reduced data
reduced_data = pd.DataFrame(reduced_data, columns = ['Dimension 1', 'Dimension 2'])

### Observation
Run the code below to see how the log-transformed sample data has changed after having a PCA transformation applied to it using only two dimensions. Observe how the values for the first two dimensions remains unchanged when compared to a PCA transformation in six dimensions.

In [57]:
# Display sample log-data after applying PCA transformation in two dimensions
print(pd.DataFrame(np.round(pca_samples, 4), columns = ['Dimension 1', 'Dimension 2']))

   Dimension 1  Dimension 2
0       1.7793      -0.6604
1       3.4131       0.1632
2      -1.9839       0.3526


## Clustering

In this section, you will choose to use either a K-Means clustering algorithm or a Gaussian Mixture Model clustering algorithm to identify the various customer segments hidden in the data. You will then recover specific data points from the clusters to understand their significance by transforming them back into their original dimension and scale. 

### Question 6
*What are the advantages to using a K-Means clustering algorithm? What are the advantages to using a Gaussian Mixture Model clustering algorithm? Given your observations about the wholesale customer data so far, which of the two algorithms will you use and why?*

**Answer:**
Clustering is a techniques used to identify 'clusters' or groups of points in the data set. Allows you to visualize relatioship between points. The K-Means algorithm works in two steps, its assigns and optimizes. K Means optimizes by the min total quadratic distance of samples to the cluster center. K - Means chooses center randomly at the start, and then optimizes from there. The advantages to K - Means is that it is linearm, and very fast O(N). Workes well with circular data. A problem is that it starts with random choice, not very repeatable or conistsient. K Means is a local minimum algorithm. 

The guassian mixture model will assign probablilities to each point being in a spefic cluster, instead of assigning  a clear label. The problem with this algorithm is that it is not as easy or trivial as K Means, and is must higher cost. This algorithm relies on Expectation Maximization. Good for when you think there are hidden paramenters. 

I would go with K - Means Clustering, because I don't have any clear indications from the 'good_data' that say go with one or the other. I choose K - Means because it is easier to implement and understand. 

### Implementation: Creating Clusters
Depending on the problem, the number of clusters that you expect to be in the data may already be known. When the number of clusters is not known *a priori*, there is no guarantee that a given number of clusters best segments the data, since it is unclear what structure exists in the data — if any. However, we can quantify the "goodness" of a clustering by calculating each data point's *silhouette coefficient*. The [silhouette coefficient](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) for a data point measures how similar it is to its assigned cluster from -1 (dissimilar) to 1 (similar). Calculating the *mean* silhouette coefficient provides for a simple scoring method of a given clustering.

In the code block below, you will need to implement the following:
 - Fit a clustering algorithm to the `reduced_data` and assign it to `clusterer`.
 - Predict the cluster for each data point in `reduced_data` using `clusterer.predict` and assign them to `preds`.
 - Find the cluster centers using the algorithm's respective attribute and assign them to `centers`.
 - Predict the cluster for each sample data point in `pca_samples` and assign them `sample_preds`.
 - Import sklearn.metrics.silhouette_score and calculate the silhouette score of `reduced_data` against `preds`.
   - Assign the silhouette score to `score` and print the result.

In [58]:
from sklearn.cluster import KMeans
# TODO: Apply your clustering algorithm of choice to the reduced data 
clusterer = KMeans(n_clusters=2, random_state=42)
clusterer.fit(reduced_data)

# TODO: Predict the cluster for each data point
preds = clusterer.predict(reduced_data)

# TODO: Find the cluster centers
centers = clusterer.cluster_centers_

# TODO: Predict the cluster for each transformed sample data point
reduced_samples = pd.DataFrame(pca_samples, columns = ['Dimension 1', 'Dimension 2'])
sample_preds = clusterer.predict(reduced_samples)

# TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
from sklearn.metrics import silhouette_score
score = silhouette_score(reduced_data, clusterer.labels_, metric='euclidean')
print("KMeans score", score)

KMeans score 0.447157742293


In [59]:
# for i in range(2,7):
#     clusterer = KMeans(n_clusters=i, random_state=42)
#     clusterer.fit(reduced_data)

#     # TODO: Predict the cluster for each data point
#     preds = clusterer.predict(reduced_data)

#     # TODO: Find the cluster centers
#     centers = clusterer.cluster_centers_

#     # TODO: Predict the cluster for each transformed sample data point
#     reduced_samples = pd.DataFrame(pca_samples, columns = ['Dimension 1', 'Dimension 2'])
#     sample_preds = clusterer.predict(reduced_samples)

#     # TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
#     score = silhouette_score(reduced_data, clusterer.labels_, metric='euclidean')
#     print("Clusters: ", i, "KMeans score", score)
#Clusters:  2 KMeans score 0.447157742293
#Clusters:  3 KMeans score 0.36398647984
#Clusters:  4 KMeans score 0.331150954285
#Clusters:  5 KMeans score 0.353193403287
#Clusters:  6 KMeans score 0.363712346094

### Question 7
*Report the silhouette score for several cluster numbers you tried. Of these, which number of clusters has the best silhouette score?* 

**Answer:**
In this reduced data set, 2 clusters had the best silhouette score. I tried up to six, and scores were lower that 2 clusters. 

### Cluster Visualization
Once you've chosen the optimal number of clusters for your clustering algorithm using the scoring metric above, you can now visualize the results by executing the code block below. Note that, for experimentation purposes, you are welcome to adjust the number of clusters for your clustering algorithm to see various visualizations. The final visualization provided should, however, correspond with the optimal number of clusters. 

In [60]:
# Display the results of the clustering from implementation
rs.cluster_results(reduced_data, preds, centers, pca_samples)

NameError: name 'rs' is not defined

### Implementation: Data Recovery
Each cluster present in the visualization above has a central point. These centers (or means) are not specifically data points from the data, but rather the *averages* of all the data points predicted in the respective clusters. For the problem of creating customer segments, a cluster's center point corresponds to *the average customer of that segment*. Since the data is currently reduced in dimension and scaled by a logarithm, we can recover the representative customer spending from these data points by applying the inverse transformations.

In the code block below, you will need to implement the following:
 - Apply the inverse transform to `centers` using `pca.inverse_transform` and assign the new centers to `log_centers`.
 - Apply the inverse function of `np.log` to `log_centers` using `np.exp` and assign the true centers to `true_centers`.


In [61]:
# TODO: Inverse transform the centers
log_centers = pca.inverse_transform(centers)

# TODO: Exponentiate the centers
true_centers = np.exp(log_centers)

# Display the true centers
segments = ['Segment {}'.format(i) for i in range(0,len(centers))]
true_centers = pd.DataFrame(np.round(true_centers), columns = df.keys())
true_centers.index = segments
print(true_centers)

            Fresh    Milk  Grocery  Frozen  Detergents_Paper  Delicatessen
Segment 0  9451.0  1938.0   2449.0  2200.0             307.0         771.0
Segment 1  5424.0  7780.0  11532.0  1123.0            4444.0        1136.0


In [62]:
np.log(true_centers)

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
Segment 0,9.153876,7.569412,7.803435,7.696213,5.726848,6.647688
Segment 1,8.598589,8.959312,9.352881,7.023759,8.399310,7.035269


### Question 8
Consider the total purchase cost of each product category for the representative data points above, and reference the statistical description of the dataset at the beginning of this project. *What set of establishments could each of the customer segments represent?*  
**Hint:** A customer who is assigned to `'Cluster X'` should best identify with the establishments represented by the feature set of `'Segment X'`.

**Answer:**
After data analysis, it seems the results give a more clear idea of what the customers purchase. Now We can see that customers that buy alot of Fresh, don't buy alot of Detergents_Papper, but ones that buy milk, grocery, seem to buy more detergent and papper.  
Segement 0 could be a market, or a small fresh food store. 
Segement 1 is probably a cafe or coffee shop as it spends alot on Milk and DEtergemnts relative to othere segement. 

### Question 9
*For each sample point, which customer segment from* ***Question 8*** *best represents it? Are the predictions for each sample point consistent with this?*

Run the code block below to find which cluster each sample point is predicted to be.

In [63]:
# Display the predictions
for i, pred in enumerate(sample_preds):
    print("Sample point", i, "predicted to be in Cluster", pred)

Sample point 0 predicted to be in Cluster 1
Sample point 1 predicted to be in Cluster 1
Sample point 2 predicted to be in Cluster 0


**Answer:**
My initial predictions of market, restaurant, market for this customer respectivelly, were not too far off, but are more refined one isntead of guessing between many options. They are narrowed down to two customer segemnts. 

From above:

      Fresh      Milk    Grocery    Frozen  Detergents_Paper  Delicatessen
0  9.446913  9.175335   8.930759  5.365976          7.891331      7.198931
1  9.409601  9.449121  10.259062  6.767343          9.395491      6.916715
2  9.175542  7.687997   7.554335  8.637462          5.370638      5.777652

	Fresh	Milk	Grocery	Frozen	Detergents_Paper	Delicatessen
Segment 0	9.153876	7.569412	7.803435	7.696213	5.726848	6.647688
Segment 1	8.598589	8.959312	9.352881	7.023759	8.399310	7.035269

As seen above at the start, the initial predictions of the samples would fall within the origional predictions. 

## Conclusion

In this final section, you will investigate ways that you can make use of the clustered data. First, you will consider how the different groups of customers, the ***customer segments***, may be affected differently by a specific delivery scheme. Next, you will consider how giving a label to each customer (which *segment* that customer belongs to) can provide for additional features about the customer data. Finally, you will compare the ***customer segments*** to a hidden variable present in the data, to see whether the clustering identified certain relationships.

### Question 10
Companies will often run [A/B tests](https://en.wikipedia.org/wiki/A/B_testing) when making small changes to their products or services to determine whether making that change will affect its customers positively or negatively. The wholesale distributor is considering changing its delivery service from currently 5 days a week to 3 days a week. However, the distributor will only make this change in delivery service for customers that react positively. *How can the wholesale distributor use the customer segments to determine which customers, if any, would reach positively to the change in delivery service?*  
**Hint:** Can we assume the change affects all customers equally? How can we determine which group of customers it affects the most?

**Answer:**
My recomendations would be different based on the segemnts. Obviously, Segement 0 spends alot of money on fresh food, therefore the 3 day delievry service would not be ideal as, fresh food is very important to be delivered everyday. These customers have a high chance of reacting negativly to this change. 

Although segment 1, is much less likely to be affect by this change as much less of thier spending is on fresh, and more on other item that arn't necessarily affacted by time. 

Based on the above results, AB testing can be used by trying the new deliverty system with system, and not on segement 0 and see how the change affects. Based on the clusters, we would think that the change would negativelty effect segemnt 1, and indifferently affter segment 2. Cost can be saved with segment 2, but segment 1 might reutrns might outweight the benefit. 

### Question 11
Additional structure is derived from originally unlabeled data when using clustering techniques. Since each customer has a ***customer segment*** it best identifies with (depending on the clustering algorithm applied), we can consider *'customer segment'* as an **engineered feature** for the data. Assume the wholesale distributor recently acquired ten new customers and each provided estimates for anticipated annual spending of each product category. Knowing these estimates, the wholesale distributor wants to classify each new customer to a ***customer segment*** to determine the most appropriate delivery service.  
*How can the wholesale distributor label the new customers using only their estimated product spending and the* ***customer segment*** *data?*  
**Hint:** A supervised learner could be used to train on the original customers. What would be the target variable?

**Answer:**
Either the distribute can run the traning and algorithm again to assign labels to the new customers. Or based off of previously results, the distributor can assign the customers into segments based off what other similar customers already in the database. 

### Visualizing Underlying Distributions

At the beginning of this project, it was discussed that the `'Channel'` and `'Region'` features would be excluded from the dataset so that the customer product categories were emphasized in the analysis. By reintroducing the `'Channel'` feature to the dataset, an interesting structure emerges when considering the same PCA dimensionality reduction applied earlier to the original dataset.

Run the code block below to see how each data point is labeled either `'HoReCa'` (Hotel/Restaurant/Cafe) or `'Retail'` the reduced space. In addition, you will find the sample points are circled in the plot, which will identify their labeling.

In [ ]:
# Display the clustering results based on 'Channel' data
rs.channel_results(reduced_data, outliers, pca_samples)

### Question 12
*How well does the clustering algorithm and number of clusters you've chosen compare to this underlying distribution of Hotel/Restaurant/Cafe customers to Retailer customers? Are there customer segments that would be classified as purely 'Retailers' or 'Hotels/Restaurants/Cafes' by this distribution? Would you consider these classifications as consistent with your previous definition of the customer segments?*

**Answer:**
It seems there is good relevenace and conistsnt with my previous definition of customer segments. Alghough it is not exact, it adds to what I thought about the customer segments. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.